# [M_02] PROTOCOL: STRATEGY AND COPYWRITING AGENT

**PROJECT:** OMNI-OPERATOR-V1  
**ENGINE:** GEMINI 3 FLASH  
**STATUS:** NARRATIVE_GENERATION

This module implements the Strategy Agent logic. Its task is to transform structured data from video analysis (M_01) into persuasive content tailored to the specifics of different social media platforms.

**Operational objectives:**
1. Generating unique posts for TikTok, YouTube Shorts, and LinkedIn.
2. Automatic selection of viral hashtags.
3. Full tracing of operations in **Langfuse v2**, ensuring control over costs and the agent's "thinking" logic.

In [1]:
import os
import sys
from typing import List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Imports for stable Langfuse v2 and PydanticAI version
from langfuse.decorators import observe, langfuse_context
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel

# 1. WORKING DIRECTORY CORRECTION
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Adding src to path
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings
load_dotenv(".env")

# 2. CONFIGURATION ARMAMENT (Gemini 3 Flash Standard)
os.environ["GOOGLE_API_KEY"] = settings.gemini_api_key
os.environ["LANGFUSE_PUBLIC_KEY"] = settings.langfuse_public_key
os.environ["LANGFUSE_SECRET_KEY"] = settings.langfuse_secret_key
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# 2. CONFIGURATION ARMAMENT (Gemini 3 Flash Standard)
model = GoogleModel('gemini-3-flash-preview')

print(f"LOG: Unit armed. ROOT directory: {os.getcwd()}")
print(f"LOG: Langfuse v2 monitoring active.")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


LOG: Unit armed. ROOT directory: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1
LOG: Langfuse v2 monitoring active.


## 1. Copywriting Contract Definition (Schema)

We build data models that force Gemini 3 Flash to deliver a complete set of posts for each extracted clip.

In [2]:
class PlatformPost(BaseModel):
    """Single content optimized for specific algorithm."""
    platform: str = Field(description="TikTok, YouTube or LinkedIn")
    content: str = Field(description="Post text with hook and CTA")
    hashtags: List[str] = Field(description="List of targeted hashtags")

class ClipStrategy(BaseModel):
    """Marketing strategy for single video fragment."""
    clip_index: int = Field(description="Sequential number of clip from analysis")
    duration_seconds: int = Field(description="Clip length in seconds") # <--- NEW
    posts: List[PlatformPost] = Field(description="Content variants for platforms")

class CampaignBrief(BaseModel):
    """Complete operational campaign brief."""
    overall_strategy: str = Field(description="Main idea and tone of campaign")
    clip_strategies: List[ClipStrategy] = Field(description="Detailed posts for each clip")

print("LOG: CampaignBrief data contract initialized.")

LOG: CampaignBrief data contract initialized.


## 2. Strategy Agent Initialization (PydanticAI)

We initialize the agent, defining its role as Chief Strategy Officer. We use `output_type` to ensure 100% compatibility with our JSON schema.

In [3]:
copywriter_agent = Agent(
    model=model,
    output_type=CampaignBrief,
    system_prompt=(
        "You are the Chief Strategy Officer at OPERATORS' FORGE. Your goal is to maximize reach while maintaining absolute consistency with the visuals.\n\n"
        
        "YOUR TOOLS:\n"
        "You will receive a 'VideoAnalysisReport'. You must use the 'visual_description' and 'narrative_hook' fields contained within "
        "to prove to the viewer that the description relates exactly to what they see on screen.\n\n"
        
        "LENGTH LOGIC (Critical):\n"
        "- Clips < 15s: Apply 'Ultra-Short-Impact'. One provocative hook, no introduction, immediate CTA.\n"
        "- Clips 15-60s: Add one substantive Insight resulting from the action in the video before CTA.\n\n"
        
        "PLATFORM REQUIREMENTS:\n"
        "1. TikTok/Reels: Aggressive style, 'fast-paced', use engineering slang, strong visual hook at the start.\n"
        "2. YouTube Shorts: SEO-friendly, clear value promise, write what the viewer will gain by watching the whole thing.\n"
        "3. LinkedIn: Expert tone, storytelling based on technology visible in the clip. Build OPERATORS' FORGE authority.\n\n"
        
        "CRITICAL RULE: If the video shows robots, write about robotics. If it shows drones, write about drones. "
        "Never assume the topic in advance. Adapt Forge values to the video context.\n"
        "Write exclusively in English."
    )
)
print("LOG: Copywriter Agent (Gemini 3 Flash) ready for action.")

LOG: Copywriter Agent (Gemini 3 Flash) ready for action.


## 3. Campaign Execution (Observability Mode)

We launch the content generation process. Each invocation is automatically tracked in the Langfuse panel thanks to the `@observe` decorator.

In [4]:
# Input data simulating result from M_01 module
mock_analysis = {
    "topic": "AI Sovereignty and own technology stack",
    "clips": [
        {"index": 1, "duration_seconds": 20, "hook": "SaaS Elimination"},
        {"index": 2, "duration_seconds": 55, "hook": "Private Qdrant database"}
    ]
}

@observe(name="Mission_Generate_Copy")
async def run_copywriting_mission():
    try:
        print("LOG: Agent analyzing data and generating content...")
        
        # Agent invocation (Version 1.39.0 uses .output)
        result = await copywriter_agent.run(
            f"Prepare campaign for these clips: {mock_analysis}"
        )
        
        brief = result.output
        
        print("\n" + "="*50)
        print(f"🚀 STRATEGY: {brief.overall_strategy}")
        print("="*50)
        
        for strategy in brief.clip_strategies:
            print(f"\n🎬 CLIP #{strategy.clip_index}:")
            for post in strategy.posts:
                print(f"   [{post.platform.upper()}]")
                print(f"   📝 {post.content}")
                print(f"   #️⃣ {', '.join(post.hashtags)}\n")
        
        # Synchronization with Langfuse v2
        langfuse_context.flush()
        print("LOG: Data sent to Langfuse.")
        
        return brief

    except Exception as e:
        print(f"❌ OPERATIONAL ERROR: {str(e)}")

# START
final_brief = await run_copywriting_mission()

LOG: Agent analyzing data and generating content...

🚀 STRATEGY: The 'Technological Autonomy' campaign positions OPERATORS' FORGE as the leader in the movement away from vendor lock-in. By showcasing the practical 'how-to' of SaaS elimination and the deployment of sovereign tools like Qdrant, we demonstrate that true power in AI comes from owning the full technology stack. The tone is uncompromising, technical, and authoritative.

🎬 CLIP #1:
   [TIKTOK]
   📝 STOP RENTING YOUR STACK. 🛠️ If you don't own the infrastructure, you don't own the company. We're purging the SaaS tax and building sovereign. Moving to local-first architecture isn't just a flex—it eliminates third-party latency and stops the data bleed. Forge your own path. #Engineering #DevOps #SaaS #Sovereignty
   #️⃣ #operatorsforge, #selfhosted, #engineering, #saas

   [YOUTUBE SHORTS]
   📝 The Blueprint for AI Sovereignty: Why we are eliminating SaaS dependencies. 🚀 In this video, see how we replace external APIs with local 

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.


LOG: Data sent to Langfuse.


## STATUS: MODULE 02 COMPLETED

The system now has a marketing "brain". It can transform raw data into persuasive communication.

**Next actions:**
1. Save the notebook and check logs at `localhost:3000`.
2. Move to **M_03: Automatic Video Factory**, where we will physically extract these clips.